In [72]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from datetime import datetime,timedelta

import pathlib

from sklearn.decomposition import PCA

# Cte
BASE_PATH = ".\in"

In [73]:
def save_to_csv(df, file_name="out.csv"):
    """
    Saves a dataframe in a csv with the name
    "out.csv" in "out" directory.
    Input:
        df
    """
    crnt_dir = pathlib.Path(__file__).parent.resolve()
    out_dir = crnt_dir/'out'
    out_dir.mkdir(parents=True, exist_ok=True)
    df.to_csv(out_dir/file_name, index=False)

In [74]:
def stretch_to_daybyday(df):
    df_tmp = df.copy(deep=True)

    row_iterator = df.iterrows()
    _, last = next(row_iterator)  # take first item from row_iterator
    for i, row in row_iterator:
        try:
            date_diff = (row.date - last.date).days
            # duplace row for the difference
            tmp =  pd.DataFrame([last])
            tmp =  pd.concat([tmp]* (date_diff-1), ignore_index=True)

            # update the date or each to make it increase
            for j, tmp_row in tmp.iterrows():
                tmp.loc[j, 'date']= tmp_row.date + timedelta(days=j+1)
                

            df_tmp = pd.concat([df_tmp,  tmp], ignore_index=True)

            last = row
        except:
            last = row

    now = datetime.now()
    if(last.date != now ):
        date_diff = (now - last.date).days
        # duplace row for the difference
        tmp =  pd.DataFrame([last])
        tmp =  pd.concat([tmp]* (date_diff-1), ignore_index=True)

        # update the date or each to make it increase
        for j, tmp_row in tmp.iterrows():
            tmp.loc[j, 'date']= tmp_row.date + timedelta(days=j+1)
            

        df_tmp = pd.concat([df_tmp,  tmp], ignore_index=True)

    df = df_tmp.sort_values(by='date', ignore_index=True )

    return df

# Economic Index Ca

In [75]:
CA_interest_rate_df = pd.read_csv(f"{BASE_PATH}\CA\CA_Bank_Rate.csv")

CA_interest_rate_df['date'] = pd.to_datetime(CA_interest_rate_df['date'], format='%B %Y')

CA_interest_rate_df = CA_interest_rate_df.loc[CA_interest_rate_df['bank_rate'] != '..']

CA_interest_rate_df = CA_interest_rate_df.sort_values(by='date')

CA_interest_rate_df.head()

,date,bank_rate
12,1935-01-01,2.50
13,1935-02-01,2.50
14,1935-03-01,2.50
15,1935-04-01,2.50
16,1935-05-01,2.50


In [76]:
CA_interest_rate_df = stretch_to_daybyday(CA_interest_rate_df)

CA_interest_rate_df.head()

,date,bank_rate
0,1935-01-01,2.50
1,1935-01-02,2.50
2,1935-01-03,2.50
3,1935-01-04,2.50
4,1935-01-05,2.50


In [77]:
bcpiWeekly_df = pd.read_csv(f"{BASE_PATH}\CA\BCPI_WEEKLY.csv",  skiprows=17)

bcpiWeekly_df['date'] = pd.to_datetime(bcpiWeekly_df['date'], format='%Y-%m-%d')

bcpiWeekly_df = bcpiWeekly_df.sort_values(by='date')


bcpiWeekly_df.head(-5)

,date,W.BCPI,W.BCNE,W.ENER,W.MTLS,W.FOPR,W.AGRI,W.FISH
0,1972-01-12,100.00,100.00,100.00,100.00,100.00,100.00,100.00
1,1972-01-19,100.00,100.00,100.00,100.00,100.00,100.00,100.00
2,1972-01-26,100.00,100.00,100.00,100.00,100.00,100.00,100.00
3,1972-02-02,100.16,100.21,99.91,100.29,100.05,100.48,95.56
4,1972-02-09,100.40,100.53,99.78,100.74,100.12,101.20,88.89
...,...,...,...,...,...,...,...,...
2642,2022-08-31,742.61,479.82,1833.19,812.06,516.34,355.10,1605.17
2643,2022-09-07,696.75,467.52,1679.26,789.81,508.23,341.34,1722.21
2644,2022-09-14,703.86,473.42,1693.73,798.47,532.23,337.18,1722.21
2645,2022-09-21,684.29,464.59,1636.80,788.80,504.64,336.74,1722.21


In [78]:
bcpiWeekly_df = stretch_to_daybyday(bcpiWeekly_df)


bcpiWeekly_df.head()

,date,W.BCPI,W.BCNE,W.ENER,W.MTLS,W.FOPR,W.AGRI,W.FISH
0,1972-01-12,100.0,100.0,100.0,100.0,100.0,100.0,100.0
1,1972-01-13,100.0,100.0,100.0,100.0,100.0,100.0,100.0
2,1972-01-14,100.0,100.0,100.0,100.0,100.0,100.0,100.0
3,1972-01-15,100.0,100.0,100.0,100.0,100.0,100.0,100.0
4,1972-01-16,100.0,100.0,100.0,100.0,100.0,100.0,100.0


In [80]:
COCWeights_df = pd.read_csv(f"{BASE_PATH}\CA\Crude_Oil_Component_Weights.csv",  skiprows=13)[:-3]

COCWeights_df['date'] = pd.to_datetime(COCWeights_df['date'], format='%Y-%m-%d')

COCWeights_df.head()

,date,WGTS.AGRI,WGTS.BRENT,WGTS.COAL,WGTS.FISH,WGTS.FOPR,WGTS.MTLS,WGTS.NATURALGAS,WGTS.WCC,WGTS.WTI
0,1972-01-01,30.0966438349,0.1339349726,1.1830782638,0.9205453099,30.4445473220,20.6185714306,2.9261469058,4.4042025519,9.2723294084
1,1973-01-01,33.7525616330,0.1373830643,1.0110418989,1.0107277333,28.4805362026,19.1857513747,2.3933703708,4.5175866363,9.5110410861
2,1974-01-01,30.1812160543,0.1716956415,1.2944243234,0.6878843365,27.6552621229,19.5158423862,2.9612795047,5.6458919411,11.8865036894
3,1975-01-01,31.3569185412,0.1719912842,2.3595358547,0.5520468188,23.1555884234,18.4900389850,6.3512954368,5.6556136003,11.9069710556
4,1976-01-01,27.6891121154,0.1634566927,2.2248001630,0.7647244924,26.1170564584,17.4992503426,8.8505097392,5.3749694271,11.3161205692


In [81]:
COCWeights_df = stretch_to_daybyday(COCWeights_df)

COCWeights_df

,date,WGTS.AGRI,WGTS.BRENT,WGTS.COAL,WGTS.FISH,WGTS.FOPR,WGTS.MTLS,WGTS.NATURALGAS,WGTS.WCC,WGTS.WTI
0,1972-01-01,30.0966438349,0.1339349726,1.1830782638,0.9205453099,30.4445473220,20.6185714306,2.9261469058,4.4042025519,9.2723294084
1,1972-01-02,30.0966438349,0.1339349726,1.1830782638,0.9205453099,30.4445473220,20.6185714306,2.9261469058,4.4042025519,9.2723294084
2,1972-01-03,30.0966438349,0.1339349726,1.1830782638,0.9205453099,30.4445473220,20.6185714306,2.9261469058,4.4042025519,9.2723294084
3,1972-01-04,30.0966438349,0.1339349726,1.1830782638,0.9205453099,30.4445473220,20.6185714306,2.9261469058,4.4042025519,9.2723294084
4,1972-01-05,30.0966438349,0.1339349726,1.1830782638,0.9205453099,30.4445473220,20.6185714306,2.9261469058,4.4042025519,9.2723294084
...,...,...,...,...,...,...,...,...,...,...
18567,2022-11-01,11.2539490053,1.5051588893,2.1633848134,0.9023752202,9.5608558829,17.1315224059,6.7587836141,24.7020281208,26.0219420483
18568,2022-11-02,11.2539490053,1.5051588893,2.1633848134,0.9023752202,9.5608558829,17.1315224059,6.7587836141,24.7020281208,26.0219420483
18569,2022-11-03,11.2539490053,1.5051588893,2.1633848134,0.9023752202,9.5608558829,17.1315224059,6.7587836141,24.7020281208,26.0219420483
18570,2022-11-04,11.2539490053,1.5051588893,2.1633848134,0.9023752202,9.5608558829,17.1315224059,6.7587836141,24.7020281208,26.0219420483


## Concat dataframe

In [82]:
ca_indices_df = pd.concat([ CA_interest_rate_df.set_index('date'), bcpiWeekly_df.set_index('date'),COCWeights_df.set_index('date')], axis=1, join='inner')
# ca_indices_df = pd.concat([ CA_interest_rate_df.set_index('date'), bcpiWeekly_df.set_index('date')], axis=1, join='inner')
ca_indices_df.head(-5)

,bank_rate,W.BCPI,W.BCNE,W.ENER,W.MTLS,W.FOPR,W.AGRI,W.FISH,WGTS.AGRI,WGTS.BRENT,WGTS.COAL,WGTS.FISH,WGTS.FOPR,WGTS.MTLS,WGTS.NATURALGAS,WGTS.WCC,WGTS.WTI
date,,,,,,,,,,,,,,,,,
1972-01-12,4.75,100.00,100.00,100.00,100.00,100.00,100.00,100.00,30.0966438349,0.1339349726,1.1830782638,0.9205453099,30.4445473220,20.6185714306,2.9261469058,4.4042025519,9.2723294084
1972-01-13,4.75,100.00,100.00,100.00,100.00,100.00,100.00,100.00,30.0966438349,0.1339349726,1.1830782638,0.9205453099,30.4445473220,20.6185714306,2.9261469058,4.4042025519,9.2723294084
1972-01-14,4.75,100.00,100.00,100.00,100.00,100.00,100.00,100.00,30.0966438349,0.1339349726,1.1830782638,0.9205453099,30.4445473220,20.6185714306,2.9261469058,4.4042025519,9.2723294084
1972-01-15,4.75,100.00,100.00,100.00,100.00,100.00,100.00,100.00,30.0966438349,0.1339349726,1.1830782638,0.9205453099,30.4445473220,20.6185714306,2.9261469058,4.4042025519,9.2723294084
1972-01-16,4.75,100.00,100.00,100.00,100.00,100.00,100.00,100.00,30.0966438349,0.1339349726,1.1830782638,0.9205453099,30.4445473220,20.6185714306,2.9261469058,4.4042025519,9.2723294084
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-10-27,3.50,658.19,465.53,1533.97,775.82,520.11,339.04,1722.21,11.2539490053,1.5051588893,2.1633848134,0.9023752202,9.5608558829,17.1315224059,6.7587836141,24.7020281208,26.0219420483
2022-10-28,3.50,658.19,465.53,1533.97,775.82,520.11,339.04,1722.21,11.2539490053,1.5051588893,2.1633848134,0.9023752202,9.5608558829,17.1315224059,6.7587836141,24.7020281208,26.0219420483
2022-10-29,3.50,658.19,465.53,1533.97,775.82,520.11,339.04,1722.21,11.2539490053,1.5051588893,2.1633848134,0.9023752202,9.5608558829,17.1315224059,6.7587836141,24.7020281208,26.0219420483


## CA PCA

In [83]:
# pca = PCA(n_components='mle')
# indices_df['pca_vec'] = pca.fit_transform(indices_df.to_numpy()).tolist()
# indices_df.pca_vec

In [84]:
ca_indices_df.to_csv("./out/CA_indices.csv", index=True)

In [118]:
ca_indices_df.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 18253 entries, 1972-01-12 to 2022-01-01
Freq: D
Data columns (total 17 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   bank_rate        18253 non-null  object 
 1   W.BCPI           18253 non-null  float64
 2   W.BCNE           18253 non-null  float64
 3   W.ENER           18253 non-null  float64
 4   W.MTLS           18253 non-null  float64
 5   W.FOPR           18253 non-null  float64
 6   W.AGRI           18253 non-null  float64
 7   W.FISH           18253 non-null  float64
 8   WGTS.AGRI        18253 non-null  object 
 9   WGTS.BRENT       18253 non-null  object 
 10  WGTS.COAL        18253 non-null  object 
 11  WGTS.FISH        18253 non-null  object 
 12  WGTS.FOPR        18253 non-null  object 
 13  WGTS.MTLS        18253 non-null  object 
 14  WGTS.NATURALGAS  18253 non-null  object 
 15  WGTS.WCC         18253 non-null  object 
 16  WGTS.WTI         18253 non-null  

# Economic Index US

In [24]:
US_interest_rate_df = pd.read_csv(f"{BASE_PATH}\\US\\US_Bank_Rate.csv")

US_interest_rate_df['date'] = pd.to_datetime(US_interest_rate_df['date'], format='%Y-%m-%d')

US_interest_rate_df = US_interest_rate_df.loc[US_interest_rate_df['bank_rate'] != '.']

US_interest_rate_df = US_interest_rate_df.sort_values(by='date')

US_interest_rate_df.head(-5)

,date,bank_rate
0,1955-08-04,3.25
1,1955-08-05,3.25
2,1955-08-08,3.25
3,1955-08-09,3.25
4,1955-08-10,3.25
...,...,...
17536,2022-10-21,6.25
17537,2022-10-24,6.25
17538,2022-10-25,6.25
17539,2022-10-26,6.25


In [25]:
US_interest_rate_df = stretch_to_daybyday(US_interest_rate_df)
US_interest_rate_df.head()

,date,bank_rate
0,1955-08-04,3.25
1,1955-08-05,3.25
2,1955-08-06,3.25
3,1955-08-07,3.25
4,1955-08-08,3.25


In [26]:
import yfinance as yf

In [27]:
GSCI = yf.Ticker('GD=F')

In [28]:
GSCI_hist = GSCI.history(period="max")
GSCI_hist['date'] = GSCI_hist.index
GSCI_hist['date'] = GSCI_hist.date.dt.strftime('%Y-%m-%d')
GSCI_hist['date'] = pd.to_datetime(GSCI_hist['date'], format='%Y-%m-%d')

GSCI_hist.head()

,Open,High,Low,Close,Volume,Dividends,Stock Splits,date
Date,,,,,,,,
2006-03-15 00:00:00-05:00,436.00,436.00,436.00,436.00,0,0,0,2006-03-15
2006-03-16 00:00:00-05:00,440.00,440.00,440.00,440.00,0,0,0,2006-03-16
2006-03-17 00:00:00-05:00,440.00,440.00,440.00,440.00,0,0,0,2006-03-17
2006-03-20 00:00:00-05:00,427.50,427.50,427.50,427.50,0,0,0,2006-03-20
2006-03-21 00:00:00-05:00,431.75,431.75,431.75,431.75,0,0,0,2006-03-21


In [29]:
GSCI_hist = stretch_to_daybyday(GSCI_hist)
GSCI_hist.head(-5)

,Open,High,Low,Close,Volume,Dividends,Stock Splits,date
0,436.000000,436.000000,436.000000,436.000000,0,0,0,2006-03-15
1,440.000000,440.000000,440.000000,440.000000,0,0,0,2006-03-16
2,440.000000,440.000000,440.000000,440.000000,0,0,0,2006-03-17
3,440.000000,440.000000,440.000000,440.000000,0,0,0,2006-03-18
4,440.000000,440.000000,440.000000,440.000000,0,0,0,2006-03-19
...,...,...,...,...,...,...,...,...
6069,634.049988,643.099976,634.000000,642.599976,22,0,0,2022-10-26
6070,644.450012,645.799988,644.200012,644.200012,6,0,0,2022-10-27
6071,638.400024,638.799988,636.000000,637.000000,10,0,0,2022-10-28
6072,638.400024,638.799988,636.000000,637.000000,10,0,0,2022-10-29


In [30]:
us_indices_df = pd.concat([ US_interest_rate_df.set_index('date'), GSCI_hist.set_index('date')], axis=1, join='inner')
us_indices_df.head(-5)

,bank_rate,Open,High,Low,Close,Volume,Dividends,Stock Splits
date,,,,,,,,
2006-03-15,7.50,436.000000,436.000000,436.000000,436.000000,0,0,0
2006-03-16,7.50,440.000000,440.000000,440.000000,440.000000,0,0,0
2006-03-17,7.50,440.000000,440.000000,440.000000,440.000000,0,0,0
2006-03-18,7.50,440.000000,440.000000,440.000000,440.000000,0,0,0
2006-03-19,7.50,440.000000,440.000000,440.000000,440.000000,0,0,0
...,...,...,...,...,...,...,...,...
2022-10-25,6.25,622.000000,630.000000,622.000000,630.000000,3,0,0
2022-10-26,6.25,634.049988,643.099976,634.000000,642.599976,22,0,0
2022-10-27,6.25,644.450012,645.799988,644.200012,644.200012,6,0,0


In [32]:
us_indices_df.to_csv("./out/US_indices.csv", index=True)